<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h1 style="font-size: 32px;">Introduction</h1>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    This notebook presents and discusses how to encode the Weierstrass Elliptic Functions, namely:
    <ul>
        <li>$\wp(z) = \wp(z, g_2, g_3)$ - Weierstrass $\wp$ eliptic function for argument $z$ and characterised by lattice invariants $g_2$ and $g_3$ </li>
        <li>$\sigma(z) = \sigma(z, g_2, g_3)$ - Weierstrass $\sigma$ function for argument $z$ and characterised by lattice invariants $g_2$ and $g_3$ </li>
        <li>$\zeta(z) = \zeta(z, g_2, g_3)$ - Weierstrass $\zeta$ function for argument $z$ and characterised by lattice invariants $g_2$ and $g_3$ </li>
    </ul>

And the inverse function $\wp^{-1}$ of Wieierstrass. Additionally, definitions of helpful quantities such as:
    <ul>
        <li>$\omega_i = \omega_i (g_2, g_3)$ - half periods or lattice generators characterised by lattice invariants $g_2$ and $g_3$ </li>
        <li>$g_2, g_3$ - lattice invariants as a function of halperiods $\omega_1$ and $\omega_3$</li>
    </ul>
    
The functions presented here are equivalent to what is currently available in Wolfram Mathematica 14 software in the field of Weierstrass Elliptic Functions.

The following two libraries are required for work:
- <code style="color:  rgb(81,206,9);font-size: 14px;">mpmath</code> 
- <code style="color:  rgb(81,206,9);font-size: 14px;">numpy</code> 

The code is mostly based on definitions from the  <a href="https://dlmf.nist.gov/23" target="_blank">DLMF</a>  database, but also on <a href="https://functions.wolfram.com/EllipticFunctions" target="_blank">Wolfram</a> database and <a href="https://en.wikipedia.org/wiki/Carlson_symmetric_form" target="_blank">Wikipedia</a>
</p>
</div>


In [38]:
from mpmath import *
import numpy as np

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
We set two important parameters:

  <code style="color:  rgb(81,206,9);font-size: 14px;">mp.dps = 25</code>: Specifies the precision of calculations to 25 decimal digits. By default, the precision is set to 15 digits, but you can adjust it to your needs.</li>

  <code style="color:  rgb(81,206,9);font-size: 14px;">mp.pretty = True</code>: Enables pretty formatting of results. When this option is active, mpmath displays results in a more readable way, e.g. by rounding the numbers and using mathematical notation.
</ul>    
</div>    

In [2]:
mp.dps = 25
mp.pretty = True

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h1 style="font-size: 32px;">1. $\wp$-function</h1>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">

Based on formulas <a href="http://dlmf.nist.gov/23.6.E1" target="_blank">(23.6.1)</a> and <a href="http://dlmf.nist.gov/23.6.E5" target="_blank">(23.6.5)</a> we can write
$$
    \wp (z) = \left( \frac{\pi}{2\omega_1} \theta_3(0,q)\theta_4(0,q) \frac{\theta_2(\pi z/(2\omega_1),q)}{\theta_1(\pi z/(2\omega_1),q)}  \right)^2 + \frac{\pi^2}{12 \omega_1^2} \left( \theta_2^4(0,q) + 2\theta_4^4(0,q)\right),
$$
where $\theta_i$ are Jacobi theta functions, $\omega_i$ are half-periods, $q=e^{i\pi\tau}$, $\tau=\omega_3/\omega_1$, and 
$$
     \operatorname{Im}(\tau)>0
$$

</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h2 style="font-size: 24px;">1.1. $\omega_j$ </h2>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
Based on formulas <a href="http://dlmf.nist.gov/23.6.E16" target="_blank">(23.6.16-17)</a> for a Legendre elliptic integral of the first kind we can  have
$$
{K}^{2}=(K\left(k\right))^{2}=\omega_{1}^{2}(e_{1}-e_{3}),
$$
$$
{K}'^{2}=(K\left(k'\right))^{2}=\omega_{3}^{2}(e_{3}-e_{1}),
$$
where 
$$
    k^2 = \frac{e_2 - e_3}{e_1 - e_3}, \quad k'^2=\frac{e_1-e_2}{e_1-e_3}.
$$
So the half-periods are given by
$$
    \omega_1 = \pm \frac{1}{\sqrt{e_1-e_3}} K(k), \quad \omega_3 = \pm \frac{1}{\sqrt{e_3 - e_1}}K(k').
$$


The choice of the appropriate sign for half-periods nie ma nie jest dokładnie określony. Bierze się to wprost z ich definicji $\wp(z) = \wp(z + 2m \omega_1 + 2n\omega_3)$, gdzie $m,n\in N$. Ważne jest by zachodziło
$$
     \operatorname{Im}(\tau)>0.
$$
Więc wykożystujemu ten warunek w definicli $\omega_1$ przez co nie musimy się nim martwić w  $\omega_3$. By otrzymać $\omega_2$ wystarczy użyć relacji  <a href="http://dlmf.nist.gov/23.2.E1" target="_blank">(23.2.1)</a>:
$$
    \omega_1 + \omega_2 + \omega_3 = 0.
$$
</div>

In [3]:
def Omega1(g2, g3):
    # Coefficients of the polynomial 4*x**3 - g2*x - g3 = 0
    coeffs = [4, 0, -g2, -g3]

    # Finding the roots
    roots = np.roots(coeffs)
    
    # Sorting the roots
    roots = np.sort_complex(roots)
    e1, e2, e3 = roots

    # Calculation of parameters k^2 and k'^2
    ksqr = (e2 - e3) / (e1 - e3)
    ksqrp = (e1 - e2) / (e1 - e3)


    # Calculating omega1 and omega3
    omega1 = (1 / sqrt(e1 - e3)) * ellipk(ksqr)
    omega3 = (1 / sqrt(e3 - e1)) * ellipk( ksqrp)

    # Calculation of the Im(tau)
    Imtau = (omega3 / omega1).imag

    # Condition for selecting omega1 sign
    res = omega1 if Imtau > 0 else -omega1

    return res

In [4]:
def Omega3(g2,g3):
    # Coefficients of the polynomial 4*x**3 - g2*x - g3 = 0
    coeffs = [4, 0, -g2, -g3]

    # Finding the roots
    roots = np.roots(coeffs)
    
    # Sorting the roots
    roots = np.sort_complex(roots)
    e1, e2, e3 = roots

    # Calculation of parameter k'^2
    ksqrp = (e1-e2)/(e1-e3)

    # Calculating omega3
    omega3= (1/sqrt(e3-e1))*ellipk(ksqrp)

    return omega3

In [5]:
def Omega2(g2,g3):
    res = -Omega1(g2,g3) - Omega3(g2,g3)
    return res

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h2 style="font-size: 24px;">1.2. $g_2$, $g_3$ </h2>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">

The invariants can be calculated by using formula provided by  <a href="https://functions.wolfram.com/EllipticFunctions/WeierstrassInvariants/27/ShowAll.html" target="_blank">Wolfram</a>
$$
    g_2 = \frac{1}{12} \left( \frac{\pi}{\omega_1} \right)^4 \left[ \theta_2(0,q)^8 - \theta_3(0,q)^4 \theta_2(0,q)^4 + \theta_3(0,q)^8 \right] ,
$$
$$
    g_3 = \left( \frac{\pi}{2 \omega_1} \right)^6 \left[ \frac{8}{27} \left( \theta_2(0,q)^{12} + \theta_3(0,q)^{12} \right) - \frac{4}{9}\left( \theta_2(0,q)^4 + \theta_3(0,q)^4\right) \theta_2(0,q)^4 \theta_3(0,q)^4 \right].
$$
</div>

In [68]:
def WeierstrassInvariants(omega1, omega3):
    ttau = omega3 / omega1
    if ttau.imag<0:
        print("Im(tau)<0, change Omega1 or Omega3")
        
    q = qfrom(tau=ttau)

    # Calculation of g2 and g3
    g2 = (pi**4 / (12 * omega1**4)) * (jtheta(2, 0, q)**8 - jtheta(3, 0, q)**4 * jtheta(2, 0, q)**4 + jtheta(3, 0, q)**8)
    
    g3 = (pi / (2 * omega1))**6 * ((8/27)*(jtheta(2, 0, q)**12 + jtheta(3, 0, q)**12)-(4/9)*(jtheta(2, 0, q)**4 +jtheta(3, 0, q)**4) * jtheta(2, 0, q)**4 * jtheta(3, 0, q)**4)

    return (g2, g3)

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h2 style="font-size: 24px;">1.2. $\wp$</h2>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">

Now using the encoded half-periods and the Jacobi thet encoded in the <code style="color:  rgb(81,206,9);font-size: 14px;">mpmath</code> package, one can present a code for the Weierstrass elliptic function
</div>

In [6]:
def WeierstrassP(z,g2,g3):
    q = qfrom(tau=Omega3(g2,g3)/Omega1(g2,g3))
    FirstCase = (pi/(2*Omega1(g2,g3))) *jtheta(3, 0,q) *jtheta(4, 0, q) * jtheta(2, pi*z/(2*Omega1(g2,g3)),q)/jtheta(1, pi*z/(2*Omega1(g2,g3)),q)
    SecondCase = (1/12)*(pi/Omega1(g2,g3))**2 * (jtheta(2, 0,q)**4 + 2*jtheta(4, 0,q)**4)
    res = FirstCase**2 + SecondCase
    return res

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h2 style="font-size: 24px;">1.3. $\wp^{-1}$</h2>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">

Based on formulas  <a href="http://dlmf.nist.gov/23.6.E30" target="_blank">(23.6.30)</a> for $t=\wp(z)$ we can write
$$
    \wp^{-1}(t)=z = \frac{1}{2} \int^\infty_t \frac{du}{\sqrt{(u-e_1)(u-e_2)(u-e_3)}}.
$$
Now we can use a definition of  <a href="https://en.wikipedia.org/wiki/Carlson_symmetric_form" target="_blank">Carlos elliptic integrals</a>:
$$
    R_F (x,y,z) = \frac{1}{2} \int^\infty_0 \frac{ds}{(s+x)(s+y)(s+z)}
$$
and by making a substitutuin $s+t=u$ notice that
$$
    \wp^{-1}(t) = R_F (z-e_1,z-e_2,z-e_3)
$$

</div>

In [7]:
def InverseWeierstrassP(z,g2,g3):
    # Coefficients of the polynomial 4*x**3 - g2*x - g3 = 0
    coeffs = [4, 0, -g2, -g3]

    # Finding the roots
    roots = np.roots(coeffs)
    
    # Sorting the roots
    roots = np.sort_complex(roots)
    e1, e2, e3 = roots
    
    return elliprf(z-e1, z-e2, z-e3)

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h2 style="font-size: 24px;">1.4. $\wp'$</h2>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">

Now using formula provided by  <a href="https://functions.wolfram.com/EllipticFunctions/WeierstrassPPrime/27/02/03/0001/" target="_blank">Wolfram</a>
$$
    \wp'(z) = - \frac{\pi^3}{4\omega_1^3} \frac{\theta_2 \left( \frac{\pi z}{2\omega_1},q \right)\theta_3 \left( \frac{\pi z}{2\omega_1},q \right)\theta_4 \left( \frac{\pi z}{2\omega_1},q \right)\theta'_1 \left( 0,q \right)^3}{\theta_2 \left( 0,q \right)\theta_3 \left( 0,q \right)\theta_4 \left( 0,q \right)\theta_1 \left( \frac{\pi z}{2\omega_1},q \right)^3}
$$

</div>

In [8]:
def WeierstrassPPrime(z,g2,g3):
    q = qfrom(tau=Omega3(g2,g3)/Omega1(g2,g3))
    numerator = jtheta(2, pi*z/(2*Omega1(g2,g3)),q)*jtheta(3, pi*z/(2*Omega1(g2,g3)),q)*jtheta(4, pi*z/(2*Omega1(g2,g3)),q)*(jtheta(1, 0, q, derivative=1)**3)
    denominator = jtheta(2, 0,q)*jtheta(3, 0,q)*jtheta(4, 0,q)*(jtheta(1, pi*z/(2*Omega1(g2,g3)),q)**3)
    res = -(1/4)*(pi/Omega1(g2,g3))**3 * (numerator/denominator)
    return res

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h1 style="font-size: 32px;">2. $\sigma$-function</h1>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">

As in previous section based on formulas <a href="http://dlmf.nist.gov/23.6.E8" target="_blank">(23.6.8-9)</a> we can write
$$
    \sigma (z) = 2 \omega_1 \exp{\left( \frac{\eta_1 z^2}{2 \omega_1} \right)} \frac{\theta_1(\pi z/(2 \omega_1),q)}{\pi \theta '_1 (0,q)},
$$
where
$$
    \eta_1 = - \frac{\pi^2}{12 \omega_1} \frac{\theta_1 ''' (0,q)}{\theta_1'(0,q)}
$$

</div>

In [9]:
def eta1(g2,g3):
    q = qfrom(tau=Omega3(g2,g3)/Omega1(g2,g3))
    res = - (pi**2/(12*Omega1(g2,g3))) * (jtheta(1, 0, q, derivative=3)/jtheta(1, 0, q, derivative=1))
    return res

In [10]:
def WeierstrassSigma(z,g2,g3):
    q = qfrom(tau=Omega3(g2,g3)/Omega1(g2,g3))
    Exp = exp((eta1(g2, g3) * z**2) / (2 * Omega1(g2, g3)))
    res = 2*Omega1(g2,g3)*Exp * (jtheta(1, pi * z/(2*Omega1(g2,g3)),q)/(pi*jtheta(1, 0, q, derivative=1)))
    return res

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h1 style="font-size: 32px;">3. $\zeta$-function</h1>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">

Here also based on basic property of $\zeta$ function <a href="http://dlmf.nist.gov/23.2.E8" target="_blank">(23.2.8)</a> we can write
$$
    \zeta (z) = \frac{\sigma'(z)}{\sigma(z)} = \frac{\eta_1}{\omega_1} z + \frac{\theta'_1(\pi z/(2 \omega_1),q)}{\theta_1(\pi z/(2 \omega_1),q)} .
$$
</div>

In [11]:
def WeierstrassZeta(z,g2,g3):
    q = qfrom(tau=Omega3(g2,g3)/Omega1(g2,g3))
    firstP = eta1(g2,g3) * z/Omega1(g2,g3)
    secondP =  pi/(2*Omega1(g2,g3)) *jtheta(1,  pi * z/(2*Omega1(g2,g3)), q, derivative=1)/jtheta(1,  pi * z/(2*Omega1(g2,g3)), q)
    return firstP + secondP

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h1 style="font-size: 32px;">4. Tests</h1>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">

As a test of the definitions presented above, we will check whether they satisfy the classical formulas presented in the <a href="https://dlmf.nist.gov/23" target="_blank">DLMF</a>  database

</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h2 style="font-size: 24px;">4.1. Inverse function</h2>
</div>

In [24]:
# We choose invariants
G2 = 1.234
G3 = -4.213

# Argument
zz = 1.2321

wp = WeierstrassP(zz,G2,G3)
print("Test:", zz- InverseWeierstrassP(wp,G2,G3))

Test: (-7.509685736771685288413282e-17 - 8.424378440054159724698837e-18j)


In [25]:
# We choose invariants
G2 = 1.895312
G3 = -7.126521

# Argument
zz= 3.213 - j*0.213

wp = WeierstrassP(zz,G2,G3)
print("Test:", zz- InverseWeierstrassP(wp,G2,G3))

Test: (2.477649715664536837637235 - 0.4260000000000000511407329j)


<div style="background-color:  rgba(153, 204, 255, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(0, 128,255); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(0, 128,255);">Note 4.1.1.</h4>
    <p>
    <ul>
        Definition of $\wp^{-1}$ to be refined in the future
    </ul>            
  </p>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h2 style="font-size: 24px;">4.2. Periodicity</h2>
</div>

<div style="background-color:  rgba(224, 224, 224, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(192, 192, 192); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(160, 160, 160);">Definition 4.2.1.</h4>
    <p> For any half-periods $\omega_j$ and $\omega_k$, where $j=1,2,3$, $k=1,2,3$ and $j\neq k$ we have (Eq.  <a href="http://dlmf.nist.gov/23.2.E9" target="_blank">(23.2.9)</a>):
$$
    \wp(z+2m\omega_j + 2n\omega_k) = \wp(z)
$$
   <p>
   </div>   

In [26]:
# We choose invariants
G2 = 1.895312
G3 = -7.126521

# Argument
zz= 3.213 - j*0.213

# Half-periods
w1 = Omega1(G2,G3)
w2 = Omega2(G2,G3)
w3 = Omega3(G2,G3)

# Tests for 
n=1
m=1
test1 = WeierstrassP(zz + 2*m*w1 + 2*n*w2,G2,G3)-WeierstrassP(zz,G2,G3)
test2 = WeierstrassP(zz + 2*m*w1 + 2*n*w3,G2,G3)-WeierstrassP(zz,G2,G3)
test3 = WeierstrassP(zz + 2*m*w2 + 2*n*w3,G2,G3)-WeierstrassP(zz,G2,G3)

print("test1=",test1)
print("test2=",test2)
print("test2=",test3)

# Tests for 
n=10
m=-7
test4 = WeierstrassP(zz + 2*m*w1 + 2*n*w2,G2,G3)-WeierstrassP(zz,G2,G3)
test5 = WeierstrassP(zz + 2*m*w1 + 2*n*w3,G2,G3)-WeierstrassP(zz,G2,G3)
test6 = WeierstrassP(zz + 2*m*w2 + 2*n*w3,G2,G3)-WeierstrassP(zz,G2,G3)

print("test4=",test4)
print("test5=",test5)
print("test6=",test6)

test1= (-1.033975765691284593589261e-25 + 1.292469707114105741986576e-26j)
test2= (2.584939414228211483973152e-25 - 2.584939414228211483973152e-25j)
test2= (1.033975765691284593589261e-25 + 7.754818242684634451919456e-26j)
test4= (3.205324873642982240126709e-24 + 1.938704560671158612979864e-24j)
test5= (1.447566071967798431024965e-24 - 6.203854594147707561535565e-25j)
test6= (2.429843049374518794934763e-24 - 1.641436528034914292322952e-24j)


<div style="background-color:  rgba(224, 224, 224, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(192, 192, 192); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(160, 160, 160);">Definition 4.2.2.</h4>
    <p> For any half-period $\omega_j$, where $j=1,2,3$, we have (Eq. <a href="http://dlmf.nist.gov/23.2.E10" target="_blank">(23.2.10)</a>):
$$
    \wp'(\omega_j) = 0
$$
   <p>
   </div>   

In [27]:
# We choose invariants
G2 = 1.895312
G3 = -7.126521

# Half-periods
w1 = Omega1(G2,G3)
w2 = Omega2(G2,G3)
w3 = Omega3(G2,G3)

print("Pp(w1)=", WeierstrassPPrime(w1,G2,G3))
print("Pp(w2)=", WeierstrassPPrime(w2,G2,G3))
print("Pp(w2)=", WeierstrassPPrime(w2,G2,G3))

Pp(w1)= (-6.931760603104882576931517e-43 - 6.296721374115339483675423e-26j)
Pp(w2)= (1.22721438984159158065958e-25 + 1.316070721200402623363533e-25j)
Pp(w2)= (1.22721438984159158065958e-25 + 1.316070721200402623363533e-25j)


<div style="background-color:  rgba(224, 224, 224, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(192, 192, 192); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(160, 160, 160);">Definition 4.2.3.</h4>
    <p> The function $\zeta(z)$ is quasi-periodic and for $j=1,2,3$ satisfies (Eq.  <a href="http://dlmf.nist.gov/23.2.E11" target="_blank">(23.2.11)</a>):
    $$
        \zeta(z+2\omega_j) = \zeta(z) + 2\zeta(\omega_j)
    $$
   <p>
   </div>   

In [28]:
# We choose invariants
G2 = 1.895312
G3 = -7.126521

# Argument
zz = 3.213 - j*0.213

# Half-periods
w1 = Omega1(G2,G3)
w2 = Omega2(G2,G3)
w3 = Omega3(G2,G3)

print("Test for w1 =", WeierstrassZeta(zz+2*w1,G2,G3)-WeierstrassZeta(zz,G2,G3)-2*WeierstrassZeta(w1,G2,G3))
print("Test for w2 =", WeierstrassZeta(zz+2*w2,G2,G3)-WeierstrassZeta(zz,G2,G3)-2*WeierstrassZeta(w2,G2,G3))
print("Test for w3 =", WeierstrassZeta(zz+2*w3,G2,G3)-WeierstrassZeta(zz,G2,G3)-2*WeierstrassZeta(w3,G2,G3))

Test for w1 = (4.591554204828980849167322e-27 + 5.169878828456422967946304e-26j)
Test for w2 = (-2.584939414228211483973152e-26 + 0.0j)
Test for w3 = (-7.754818242684634451919456e-26 + 1.292469707114105741986576e-26j)


<div style="background-color:  rgba(224, 224, 224, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(192, 192, 192); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(160, 160, 160);">Definition 4.2.4.</h4>
    <p> The function $\sigma(z)$ is quasi-periodic and for $j=1,2,3$ satisfies (Eq.  <a href="http://dlmf.nist.gov/23.2.E15" target="_blank">(23.2.15)</a>):
    $$
        \sigma(z+2\omega_j) = - e^{2\eta_j (z+\omega_j)}\sigma(z), \quad \eta_j = \zeta(\omega_j)
    $$
   <p>
   </div>   

In [31]:
# We choose invariants
G2 = 1.895312
G3 = -7.126521

# Argument
zz = 3.213 - j*0.213

# Half-periods
w1 = Omega1(G2,G3)
w2 = Omega2(G2,G3)
w3 = Omega3(G2,G3)

print("Test for w1 =", WeierstrassSigma(zz+2*w1,G2,G3)+exp(2*WeierstrassZeta(w1,G2,G3)*(zz+w1))* WeierstrassSigma(zz,G2,G3))
print("Test for w2 =", WeierstrassSigma(zz+2*w2,G2,G3)+exp(2*WeierstrassZeta(w2,G2,G3)*(zz+w2))* WeierstrassSigma(zz,G2,G3))
print("Test for w3 =", WeierstrassSigma(zz+2*w3,G2,G3)+exp(2*WeierstrassZeta(w3,G2,G3)*(zz+w3))* WeierstrassSigma(zz,G2,G3))

Test for w1 = (-2.150669592637871954665663e-23 - 9.926167350636332098456904e-24j)
Test for w2 = (-3.437969420923521273684292e-24 + 0.0j)
Test for w3 = (-6.352747104407252543012419e-22 + 1.482307657695025593369564e-21j)


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h2 style="font-size: 24px;">4.3. Differential equation</h2>
</div>

<div style="background-color:  rgba(224, 224, 224, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(192, 192, 192); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(160, 160, 160);">Definition 4.3.1.</h4>
    <p> 
        Then the $\wp$-function satisfies the differential equation (Eq. <a href="http://dlmf.nist.gov/23.3.E10" target="_blank">(23.3.10)</a>)
    $$
        \wp'^2(z) = 4 \wp^3(z) - g_2 \wp(z) -g_3
    $$ 
   <p>
   </div>  

In [50]:
# We choose invariants
G2 = 1.895312
G3 = -7.126521

# Argument
zz = 3.213 - j*0.213

left = WeierstrassPPrime(zz,G2,G3)**2
right = 4*WeierstrassP(zz,G2,G3)**3 - G2*WeierstrassP(zz,G2,G3) - G3

print("Test=", left - right)

Test= (-1.614271245345283503057947e-15 + 2.562300535517916121622207e-16j)


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h2 style="font-size: 24px;">4.4. Roots</h2>
</div>

<div style="background-color:  rgba(224, 224, 224, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(192, 192, 192); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(160, 160, 160);">Definition 4.4.1.</h4>
    <p> 
    Given any half-perods $\omega_j$, $j=1,2,3$ we can identify the $e_j$ individually, via (Eq. <a href="http://dlmf.nist.gov/23.3.E9" target="_blank">23.3.9</a>)
    $$
        e_j = \wp(\omega_j)
    $$
   <p>
   </div>  

In [62]:
# We choose invariants
G2 = 1.895312
G3 = -7.126521

# Argument
zz = 3.213 - j*0.213

# Half-periods
w1 = Omega1(G2,G3)
w2 = Omega2(G2,G3)
w3 = Omega3(G2,G3)

# Coefficients of the polynomial 4*x**3 - g2*x - g3 = 0
coeffs = [4, 0, -G2, -G3]

# Finding the roots
roots = np.roots(coeffs)
    
# Sorting the roots
roots = np.sort_complex(roots)
e1, e2, e3 = roots

print("e1 - wp(w1)=", e1 - WeierstrassP(w1,G2,G3) )
print("e2 - wp(w2)=", e2 - WeierstrassP(w2,G2,G3) )
print("e3 - wp(w3)=", e3 - WeierstrassP(w3,G2,G3) )

e1 - wp(w1)= (1.324551388116140543331598e-16 - 3.49091863489540942883054e-18j)
e2 - wp(w2)= (1.777149086210981423180532e-16 + 6.981837347299436801338255e-18j)
e3 - wp(w3)= (1.324551388116140543331598e-16 - 3.490918686574415471810185e-18j)


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h2 style="font-size: 24px;">4.5. Addition formulas</h2>
</div>

<div style="background-color:  rgba(224, 224, 224, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(192, 192, 192); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(160, 160, 160);">Definition 4.5.1.</h4>
    <p>
    Let $u,v\in\mathbb{C}$, so that $u,v,u+v,u-v\notin\Lambda$. Then one has (Eq. <a href="http://dlmf.nist.gov/23.10.E1" target="_blank">23.10.1</a>):      
    $$
        \wp(u+v) = \frac{1}{4} \left( \frac{\wp'(u) - \wp'(v)}{\wp(u) - \wp(v)}\right)^2 - \wp(u) - \wp(v)
    $$ 
   <p>
   </div>  

In [57]:
# We choose invariants
G2 = 1.895312
G3 = -7.126521

# Argument
u = 3.213 - j*0.213
v = 1.986 + j*1.218

left = WeierstrassP(u + v,G2,G3)
right = (1/4)*((WeierstrassPPrime(u,G2,G3)-WeierstrassPPrime(v,G2,G3))/(WeierstrassP(u,G2,G3)-WeierstrassP(v,G2,G3)))**2 - WeierstrassP(u,G2,G3) - WeierstrassP(v,G2,G3)
print("Test=", left-right)

Test= (2.235086292545507092537055e-16 - 4.717514430966485958251003e-25j)


<div style="background-color:  rgba(224, 224, 224, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(192, 192, 192); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(160, 160, 160);">Definition 4.5.2.</h4>
    <p> 
       Let $u,v\in\mathbb{C}$, so that $u,v,u+v,u-v\notin\Lambda$. Then one has (Eq. <a href="http://dlmf.nist.gov/23.10.E2" target="_blank">23.10.2</a>):      
    $$
        \zeta(u+v) = \zeta(u) + \zeta(v) + \frac{1}{2} \frac{\zeta''(u)-\zeta''(v)}{\zeta'(u) - \zeta'(u)}
    $$ 
        Hovewer we have Eq. <a href="http://dlmf.nist.gov/23.2.E7" target="_blank">23.2.7</a> $\wp(z) = - \zeta'(z)$, so
    $$
        \zeta(u+v) = \zeta(u) + \zeta(v) + \frac{1}{2} \frac{\wp'(u)-\wp'(v)}{\wp(u) - \wp(u)}
    $$     
   <p>
   </div>  

In [59]:
# We choose invariants
G2 = 1.895312
G3 = -7.126521

# Argument
u = 3.213 - j*0.213
v = 1.986 + j*1.218

left = WeierstrassZeta(u + v,G2,G3)
right = WeierstrassZeta(u ,G2,G3) +WeierstrassZeta(v,G2,G3) + (1/2) *(WeierstrassPPrime(u,G2,G3)-WeierstrassPPrime(v,G2,G3))/(WeierstrassP(u,G2,G3)-WeierstrassP(v,G2,G3))
print("Test=", left-right)

Test= (-2.067951531382569187178522e-25 - 3.877409121342317225959728e-26j)


<div style="background-color:  rgba(224, 224, 224, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(192, 192, 192); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(160, 160, 160);">Definition 4.5.3.</h4>
    <p> 
        Let $u,v\in\mathbb{C}$, so that $u,v,u+v,u-v\notin\Lambda$. Then one has (Eq. <a href="http://dlmf.nist.gov/23.10.E3" target="_blank">23.10.3</a>):      
    $$
        \frac{\sigma(u+v)\sigma(u-v)}{\sigma^2(u)\sigma^2(v)} = \wp(v) - \wp(u)
    $$ 
   <p>
   </div>  

In [61]:
# We choose invariants
G2 = 1.895312
G3 = -7.126521

# Argument
u = 3.213 - j*0.213
v = 1.986 + j*1.218

left = WeierstrassSigma(u + v,G2,G3)* WeierstrassSigma(u - v,G2,G3)/( WeierstrassSigma(u,G2,G3)**2 *  WeierstrassSigma(v,G2,G3)**2)
right = WeierstrassP(v,G2,G3)-WeierstrassP(u,G2,G3)
print("Test=", left-right)

Test= (1.075334796318935977332831e-23 - 3.101927297073853780767783e-24j)
